In [1]:
import pandas as pd
import numpy as np

In [18]:
history_log_df = pd.read_csv("output/int_0_trace/int_0_trace_history.log")
history_log_df.head(10)

,Key,PC,Next_PC,Pred_Cycle,Fetch_Cycle,Execute_Cycle,Pred_Dir,Taken,Predictor_Used,Src_Regs,Dst_Reg,Mem_VA,Inst_Class,GHIST,Load_Dependence
0,668266592,2.147493e+09,0.000000e+00,14704974,14704974,14704986,0,2,0,[],255,0,3,10540991632317786794,0
1,668265824,2.814747e+14,2.814747e+14,0,0,14704972,0,255,10,[21;28],17,281474976695400,1,0,0
2,668265840,2.814747e+14,2.814747e+14,0,0,14704971,0,255,10,[26;22],5,281462941627014,1,0,0
3,668265696,2.814747e+14,2.814747e+14,0,0,14704970,0,255,10,[26;22],5,281462941627014,1,0,0
4,668265552,2.814747e+14,2.814747e+14,0,0,14704970,0,255,10,[26;22],5,281462941627014,1,0,0
5,668265536,2.814747e+14,2.814747e+14,0,0,14704970,0,255,10,[21;28],17,281474976695392,1,0,0
6,668265408,2.814747e+14,2.814747e+14,0,0,14704970,0,255,10,[26;22],5,281462941627014,1,0,0
7,668265264,2.814747e+14,2.814747e+14,0,0,14704970,0,255,10,[26;22],5,281462941627014,1,0,0
8,668265120,2.814747e+14,2.814747e+14,0,0,14704970,0,255,10,[26;22],5,281462941627014,1,0,0
9,668264976,2.814747e+14,2.814747e+14,0,0,14704969,0,255,10,[26;22],5,281462941627014,1,0,0


In [21]:
history_log_df[
    # (history_log_df['Next_PC'] != 0)  
    (history_log_df['Inst_Class'] == 1)  
    # & (~history_log_df['Src_Regs'].astype(str).isin(['[]', '[64]']))
    & (history_log_df['Dst_Reg'] == 64)
].head(20)

,Key,PC,Next_PC,Pred_Cycle,Fetch_Cycle,Execute_Cycle,Pred_Dir,Taken,Predictor_Used,Src_Regs,Dst_Reg,Mem_VA,Inst_Class,GHIST,Load_Dependence


In [ ]:
# l = []
# GHR = ""
# GHR_length = 8
# history_log_joined_df['GHR'] = 0
# # print("HI")
# for index,row in history_log_joined_df.iterrows():
#     if index > 200:
#         break
#     if(row['instClass'] == 'condBranch'):
#         GHR += str(row['taken'])
#         # print(GHR,"jdfk")
#         history_log_joined_df.iloc[index,-1] = GHR[-1-GHR_length:-1]
#         # for i in range(index-1,-1,-1):
#         #     l.append(history_log_df.iloc[i,:])
#             # print(history_log_df.iloc[i,:])
# pd.DataFrame(l).head()

C:\Users\vgr48\AppData\Local\Temp\ipykernel_94636\1865889121.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  history_log_joined_df.iloc[index,-1] = GHR[-1-GHR_length:-1]


""
